In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp core

# Imports

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *


In [ ]:
#export
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python
import matplotlib
import matplotlib.pyplot as plt # for plotting


# Venn diagram
from matplotlib_venn import venn2
import re
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
eng_stopwords = stopwords.words('english')
import gc

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from fastai2.basics import *
from fastai2.text.all import *
from fastai2.callback.all import *

In [ ]:
#export
import os
print(os.listdir("../data/gquest_data/"))
data_path = Path("../data/gquest_data/")

['sample_submission.csv', 'test', 'train']


In [ ]:
import pdb

# Reading data

In [ ]:
#export
print('Reading data...')
train_data = pd.read_csv(data_path/'train/train.csv')
test_data = pd.read_csv(data_path/'test/test.csv')
sample_submission = pd.read_csv(str(data_path/'sample_submission.csv'))
print('Reading data completed')

Reading data...
Reading data completed


In [ ]:
#export
print('Size of train_data', train_data.shape)
print('Size of test_data', test_data.shape)
print('Size of sample_submission', sample_submission.shape)
test_eq(train_data.shape,(6079,41))
test_eq(test_data.shape,(476,11))
test_eq(sample_submission.shape,(476,31))

Size of train_data (6079, 41)
Size of test_data (476, 11)
Size of sample_submission (476, 31)


In [ ]:
train_data.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes instead of a macro lens?,"After playing around with macro photography on-the-cheap (read: reversed lens, rev. lens mounted on a straight lens, passive extension tubes), I would like to get further with this. The problems with the techniques I used is that focus is manual and aperture control is problematic at best. This limited my setup to still subjects (read: dead insects) Now, as spring is approaching, I want to be able to shoot live insects. I believe that for this, autofocus and settable aperture will be of great help.\n\nSo, one obvious but expensive option is a macro lens (say, EF 100mm Macro) However, I am ...",ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skinny.\n\n\n ...what am I losing when using tubes...?\n\n\nA very considerable amount of light! Increasing that distance from the end of the lens to the sensor can cut your light several stops. Combined with the fact that you'll usually shoot stopped down - expect to need to increase your ISO considerably.\n\nThe fact the macro's are usually considered very very sharp, although I believe that 70-200mm 2.8 is supposed to be quite sharp.\n\nThe ultra low distortion typical of many macros.\n\nI wouldn't worry too much about the bokeh since the DOF...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/what-am-i-losing-when-using-extension-tubes-instead-of-a-macro-lens,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a sprawl/metroplex... between downtown and a commercial district?,"I am trying to understand what kinds of places the spam values on p 231 refer to in the 5th Edition main book for Shadowrun.\n\nPer p 15, a sprawl is a plex, a plex is a ""metropolitan complex, short for metroplex"". Per Google a metroplex is "" a very large metropolitan area, especially one that is an aggregation of two or more cities"". A city downtown and sprawl downtown would tend to have similar densities, but for some reason the sprawl (which includes suburbs?) has a higher spam zone noise rating (p 231). Similarly, I'd think of a downtown as being more dense and noisy (e.g. Office bui...",russellpierce,https://rpg.stackexchange.com/users/8774,"It might be helpful to look into the definition of spam zone:\n\n(p.216) spam zone: An area flooded with invasive and/or viral AR advertising, causing noise.\n\nBecause a metroplex has so many marketing targets, it seems a safe assumption that marketers would drown the plex with spam. Spam from the less dense areas would bleed into the urban cores. A smaller city with less urban/suburban territory surrounding it ostensibly wouldn't have as much spam.\n",Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/what-is-the-distinction-between-a-city-and-a-sprawl-metroplex-between-downtow,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole component pins,"I'm working on a PCB that has through-hole components on both sides of the board. The ""top"" side of the board is mounted flush to a Delrin plastic block (the only top-side component is a gas sensor that is fed air samples through hose fittings in the plastic block).\n\nThe flush mounting means that I have to add grooves to the plastic block to accommodate the soldered pins of the bottom-side components. Assuming a standard 0.062"" thickness FR4 board, how deep do I need to make the grooves in

In [ ]:
train_data.columns

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host', 'question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfa

In [ ]:
sample_submission.columns

Index(['qa_id', 'question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written'],
      dtype='object')

In [ ]:
targets = list(sample_submission.columns[1:])
targets

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written',
 'answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [ ]:
text_columns=['question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host']

In [ ]:
train_data[targets].describe()

,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,...,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000,6079.000000
mean,0.892663,0.595301,0.057301,0.698525,0.772633,0.793689,0.587478,0.507275,0.238745,0.004469,...,0.799931,0.925408,0.654823,0.960054,0.968626,0.854680,0.479547,0.130641,0.502468,0.908254
std,0.132047,0.219470,0.182196,0.350938,0.303023,0.336622,0.135900,0.185987,0.335057,0.045782,...,0.178420,0.114836,0.107666,0.086926,0.074631,0.130743,0.422921,0.225718,0.407097,0.100708
min,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000,...,0.333333,0.333333,0.333333,0.333333,0.333333,0.200000,0.000000,0.000000,0.000000,0.333333
25%,0.777778,0.444444,0.000000,0.500000,0.666667,0.666667,0.444444,0.333333,0.000000,0.000000,...,0.666667,0.888889,0.666667,1.000000,1.000000,0.800000,0.000000,0.000000,0.000000,0.888889
50%,0.888889,0.555556,0.000000,0.666667,1.000000,1.000000,0.555556,0.444444,0.000000,0.000000,...,0.833333,1.000000,0.666667,1.000000,1.000000,0.866667,0.500000,0.000000,0.500000,0.888889
75%,1.000000,0.777778,0.000000,1.000000,1.000000,1.000000,0.666667,0.666667,0.333333,0.000000,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.933333,1.000000,0.333333,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
train_data['question_body'][0]

'After playing around with macro photography on-the-cheap (read: reversed lens, rev. lens mounted on a straight lens, passive extension tubes), I would like to get further with this. The problems with the techniques I used is that focus is manual and aperture control is problematic at best. This limited my setup to still subjects (read: dead insects) Now, as spring is approaching, I want to be able to shoot live insects. I believe that for this, autofocus and settable aperture will be of great help.\n\nSo, one obvious but expensive option is a macro lens (say, EF 100mm Macro) However, I am not really interested in yet another prime lens. An alternative is the electrical extension tubes.\n\nExcept for maximum focusing distance, what am I losing when using tubes (coupled with a fine lens, say EF70-200/2.8) instead of a macro lens?\n'

In [ ]:
text_columns

['question_title',
 'question_body',
 'question_user_name',
 'question_user_page',
 'answer',
 'answer_user_name',
 'answer_user_page',
 'url',
 'category',
 'host']

In [ ]:
df_tokenized,token_count=tokenize_df(train_data,text_columns)

In [ ]:
df_tokenized.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written,text
0,0,1.000000,0.333333,0.000000,0.0,0.0,0.0,1.000000,1.000000,0.000000,...,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000,"[xxbos, xxfld, 1, xxmaj, what, am, i, losing, when, using, extension, tubes, instead, of, a, macro, lens, ?, xxfld, 2, xxmaj, after, playing, around, with, macro, photography, on, -, the, -, cheap, (, read, :, reversed, lens, ,, rev, ., lens, mounted, on, a, straight, lens, ,, passive, extension, tubes, ), ,, i, would, like, to, get, further, with, this, ., xxmaj, the, problems, with, the, techniques, i, used, is, that, focus, is, manual, and, aperture, control, is, problematic, at, best, ., xxmaj, this, limited, my, setup, to, still, subjects, (, read, :, dead, insects, ), xxmaj, now, ,, ..."
1,1,1.000000,1.000000,0.000000,0.5,1.0,1.0,0.444444,0.444444,0.666667,...,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889,"[xxbos, xxfld, 1, xxmaj, what, is, the, distinction, between, a, city, and, a, sprawl, /, metroplex, …, between, downtown, and, a, commercial, district, ?, xxfld, 2, i, am, trying, to, understand, what, kinds, of, places, the, spam, values, on, p, 231, refer, to, in, the, 5th, xxmaj, edition, main, book, for, xxmaj, shadowrun, ., \n\n, xxmaj, per, p, 15, ,, a, sprawl, is, a, plex, ,, a, plex, is, a, "", metropolitan, complex, ,, short, for, metroplex, "", ., xxmaj, per, xxmaj, google, a, metroplex, is, "", a, very, large, metropolitan, area, ,, especially, one, that, is, an, aggregation, of, ..."
2,2,0.888889,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.444444,0.333333,...,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889,"[xxbos, xxfld, 1, xxmaj, maximum, protusion, length, for, through, -, hole, component, pins, xxfld, 2, xxmaj, i, 'm, working, on, a, xxup, pcb, that, has, through, -, hole, components, on, both, sides, of, the, board, ., xxmaj, the, "", top, "", side, of, the, board, is, mounted, flush, to, a, xxmaj, delrin, plastic, block, (, the, only, top, -, side, component, is, a, gas, sensor, that, is, fed, air, samples, through, hose, fittings, in, the, plastic, block, ), ., \n\n, xxmaj, the, flush, mounting, means, that, i, have, to, add, grooves, to, the, plastic, block, to, accommodate, the, solder..."
3,3,0.888889,0.666667,0.666667,1.0,1.0,1.0,0.444444,0.444444,0.000000,...,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000,"[xxbos, xxfld, 1, xxmaj, can, an, affidavit, be, used, in, xxmaj, beit, xxmaj, din, ?, xxfld, 2, xxmaj, an, affidavit, ,, from, what, i, understand, ,, is, basically, a, signed, document, given, by, a, witness, to, be, used, as, evidence, in, a, trial, ,, without, the, witness, themselves, needing, to, take, a, stand, ., \n\n, xxmaj, can, an, affidavit, be, used, in, xxmaj, beit, xxmaj, din, ?, xxmaj, or, must, witnesses, take, the, stand, in, person, for, their, testimony, to, count, ?, \n\n, (, in, case, i, 'm, misunderstanding, what, exactly, an, affidavit, is, ,, simply, treat, it, as,..."
4,5,1.000000,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.666667,0.000000,...,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000,"[xxbos, xxfld, 1, xxmaj, how, do, you, make, a, binary, image, in, xxmaj, photoshop, ?, xxfld, 2, i, am, trying, to, make, a, binary, image, ., i, want, more, than, just, the, look, of, the, image, to, be, black, /, white, ,, but, i, want, the, actual, file, to, be, a, binary, file, ., xxmaj, every, pixel, should, be, either, black, ,, or, white, ., \n\n, i, do, n't,

In [ ]:
vocab = make_vocab(token_count)
vocab

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 '/',
 'the',
 ',',
 '-',
 '.',
 'to',
 '\n▁',
 'a',
 ':',
 ')',
 'i',
 'is',
 '\n\n',
 'and',
 'of',
 '"',
 '(',
 'in',
 '\n',
 'it',
 'that',
 '\\',
 'you',
 '$',
 'for',
 ';',
 'this',
 '?',
 '=',
 'users',
 'https',
 '}',
 "'",
 'with',
 'on',
 'be',
 '3',
 '>',
 '1',
 'if',
 'have',
 'as',
 'not',
 'can',
 'are',
 '<',
 '2',
 '{',
 'or',
 'but',
 'do',
 '\n\n\n',
 '4',
 'http',
 '5',
 'from',
 'my',
 "'s",
 "n't",
 '10',
 '6',
 ']',
 'an',
 '7',
 '8',
 'questions',
 '9',
 '\n\n▁',
 'so',
 'how',
 'what',
 'at',
 'your',
 'there',
 'will',
 'would',
 'stackoverflow.com',
 'by',
 'use',
 'one',
 'when',
 'which',
 'like',
 'all',
 '0',
 'some',
 'does',
 '&',
 'get',
 'any',
 'using',
 'they',
 'then',
 '[',
 'was',
 '*',
 'just',
 'more',
 '+',
 'should',
 'has',
 'we',
 'other',
 'want',
 '#',
 "'m",
 'need',
 'only',
 'way',
 'no',
 'about',
 'am',
 'up',
 'time',
 'also',
 'new',
 'out',
 '

In [ ]:
tfm = Numericalize(make_vocab(token_count))

In [ ]:
splits = RandomSplitter()(df_tokenized)

In [ ]:
splits

((#4864) [4044,1020,2599,4067,2286,1358,3581,2202,3698,2611...],
 (#1215) [1992,4236,1477,5190,3561,342,4352,1956,126,1640...])

In [ ]:
dsrc = DataSource(df_tokenized, [[attrgetter("text"), tfm]], splits=splits, dl_type=LMDataLoader)

In [ ]:
bs,sl = 50,300
dbch = dsrc.databunch(bs=bs, seq_len=sl, after_batch=Cuda)

In [ ]:
dbch.show_batch()

,text,text_
0,"xxbos xxfld 1 xxmaj does this statement make any sense ? xxfld 2 i am asking this question completely out of curiosity . xxmaj the other day , my roommate , by mistake , used ' light year ' as a unit of time instead of distance . xxmaj when i corrected him ( pedantic , much ) , he said the following : \n\n "" units are relative . xxmaj and according to xxmaj fourier xxmaj transforms , units can be changed so xxmaj light year is a unit of time . "" \n\n xxmaj that got me thinking and i read up xxmaj fourier xxmaj transforms on wikipedia but could n't find anything about using a unit in one domain as a unit for another measurement . i do agree that units ( particularly , base units are relative . eg : the meter ) , but does","xxfld 1 xxmaj does this statement make any sense ? xxfld 2 i am asking this question completely out of curiosity . xxmaj the other day , my roommate , by mistake , used ' light year ' as a unit of time instead of distance . xxmaj when i corrected him ( pedantic , much ) , he said the following : \n\n "" units are relative . xxmaj and according to xxmaj fourier xxmaj transforms , units can be changed so xxmaj light year is a unit of time . "" \n\n xxmaj that got me thinking and i read up xxmaj fourier xxmaj transforms on wikipedia but could n't find anything about using a unit in one domain as a unit for another measurement . i do agree that units ( particularly , base units are relative . eg : the meter ) , but does his statement make any sense ? \n\n xxup edit \n xxmaj thank you everyone for all the answers . xxmaj it is n't so much to in it in or prove a point as it is to understand the concept better . xxmaj anyways this is his response after i showed him this thread . xxmaj any comments would be appreciated . \n\n xxmaj his response : \n xxmaj nevermind , for the first time i accept i was wrong . xxup but using lightyears to measure time is possible . xxmaj my example did n't make sense bacause i was wrong when i meantioned that xxmaj i 'm still measuring dist . xxmaj if you have a signal in time domain and … take the xxup ft , i get a signal which xxup does xxup not xxup have to be in frequency domain . xxmaj clarify this to the"
1,"n't need to run the application . \n\n xxmaj what seems wrong in your case is the deployment process . xxmaj instead of pushing the code to the production server directly from the source control , you should rather , in the simplest case : \n\n\n xxmaj load the source code , \n xxmaj compile and / or optimize it , \n xxmaj push the compiled version to the server . \n\n\n xxmaj if your website has lots of visits , you may need a more complicated flow , using two servers , for example : \n\n\n xxmaj load the source code , \n xxmaj compile and / or optimize it , \n xxmaj push the compiled version to the xxunk server , \n xxmaj run a bunch of tests to ensure that the website is working on xxunk server , \n xxmaj forward all the requests to the production","need to run the application . \n\n xxmaj what seems wrong in your case is the deployment process . xxmaj instead of pushing the code to the production server directly from the source control , you should rather , in the simplest case : \n\n\n xxmaj load the source code , \n xxmaj compile and / or optimize it , \n xxmaj push the compiled version to the server . \n\n\n xxmaj if your website has lots of visits , you may need a more complicated flow , using two servers , for example : \n\n\n xxmaj load the source code , \n xxmaj compile and / or optimize it , \n xxmaj push the compiled version to the xxunk server , \n xxmaj run a bunch of tests to ensure that the website is working on xxunk server , \n xxmaj forward all the requests to the production server 2 , \n xxmaj once the production server 1 does n't have any requests , push the compiled version to it , \n xxmaj forward all the requests to the production server 1 , \n xxmaj once the production server 2 does n't have any requests , push the xxunk version to i

In [ ]:
config = awd_lstm_lm_config.copy()
config.update({'input_p': 0.6, 'output_p': 0.4, 'weight_p': 0.5, 'embed_p': 0.1, 'hidden_p': 0.2})
model = get_language_model(AWD_LSTM, len(vocab), config=config)

In [ ]:
opt_func = partial(Adam, wd=0.1, eps=1e-7)
cb_funcs = [partial(MixedPrecision, clip=0.1), partial(RNNTrainer, alpha=2, beta=1)]

In [ ]:
learn = Learner(dbch, model, loss_func=CrossEntropyLossFlat(), opt_func=opt_func, cb_funcs=cb_funcs, metrics=[accuracy, Perplexity()])

In [ ]:
learn = language_model_learner(dbch, AWD_LSTM, metrics=[accuracy, Perplexity()], path=data_path, opt_func = partial(Adam, wd=0.1)).to_fp16()

In [ ]:
learn.freeze()
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.686468,4.034666,0.327226,56.524014,01:08


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, 1e-2, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.024105,3.719761,0.367150,41.254520,01:22
1,3.692448,3.475374,0.396708,32.309925,01:22
2,3.330445,3.289187,0.419726,26.821041,01:22
3,3.076730,3.236449,0.427728,25.443218,01:22


In [ ]:
learn.show_results()

,input,target,pred
0,"xxbos xxfld 1 xxmaj is it possible to withdraw xxup usd from atms in xxmaj tanzania ? xxfld 2 i will be xxmaj in the following xxmaj xxunk towns , xxmaj arusha , xxmaj xxunk es xxmaj xxunk and xxmaj stone xxmaj town and would like to know where i am best to xxunk xxmaj my stock of xxup usd ? \n\n xxmaj is it possible to withdraw xxup usd from atms in xxmaj tanzania or will i need to withdraw local currency and use a currency exchange ? \n▁ xxfld 3 xxmaj stuart xxfld 4 https : / / travel.stackexchange.com / users / 449 xxfld 5 i have n't been to xxmaj arusha like xxunk , but i have been to quite a few places in xxmaj tanzania and have yet to find an xxup atm in the country that gives out xxup usd . xxmaj so , do n't count on it , though there might be one or two xxunk away somewhere . \n▁ xxfld 6 xxunk xxfld 7 https : / / travel.stackexchange.com / users / xxunk xxfld 8 http : / / travel.stackexchange.com / questions / xxrep 3 9 6 / is - it - possible - to - withdraw - usd - from - atms - in - tanzania / xxunk xxfld 9 xxup culture xxfld 10 travel.stackexchange.com xxbos xxfld 1 xxmaj should i include high school details in xxmaj grad xxmaj school xxmaj resume ? xxfld 2 i was wondering if i should include my high school details in the resume of my graduate school application ? xxmaj it 's unclear about this bit since we are not actually submitting any proof of high school records during grad school application ( they only ask for undergraduate details ) . \n xxmaj but my","xxfld 1 xxmaj is it possible to withdraw xxup usd from atms in xxmaj tanzania ? xxfld 2 i will be xxmaj in the following xxmaj xxunk towns , xxmaj arusha , xxmaj xxunk es xxmaj xxunk and xxmaj stone xxmaj town and would like to know where i am best to xxunk xxmaj my stock of xxup usd ? \n\n xxmaj is it possible to withdraw xxup usd from atms in xxmaj tanzania or will i need to withdraw local currency and use a currency exchange ? \n▁ xxfld 3 xxmaj stuart xxfld 4 https : / / travel.stackexchange.com / users / 449 xxfld 5 i have n't been to xxmaj arusha like xxunk , but i have been to quite a few places in xxmaj tanzania and have yet to find an xxup atm in the country that gives out xxup usd . xxmaj so , do n't count on it , though there might be one or two xxunk away somewhere . \n▁ xxfld 6 xxunk xxfld 7 https : / / travel.stackexchange.com / users / xxunk xxfld 8 http : / / travel.stackexchange.com / questions / xxrep 3 9 6 / is - it - possible - to - withdraw - usd - from - atms - in - tanzania / xxunk xxfld 9 xxup culture xxfld 10 travel.stackexchange.com xxbos xxfld 1 xxmaj should i include high school details in xxmaj grad xxmaj school xxmaj resume ? xxfld 2 i was wondering if i should include my high school details in the resume of my graduate school application ? xxmaj it 's unclear about this bit since we are not actually submitting any proof of high school records during grad school application ( they only ask for undergraduate details ) . \n xxmaj but my high","xxfld 1 xxmaj how there possible to set xxup usd from a in xxmaj tanzania ? xxfld 2 xxmaj am be living in xxmaj xxup xxmaj tanzania xxmaj , xxmaj tanzania , xxmaj xxunk , xxmaj xxunk , xxmaj xxunk xxmaj town . xxmaj like to be if to am going to live the xxunk money of xxmaj usd . \n\n xxmaj is there possible to get xxup usd from a in xxmaj tanzania ? should i be to return money currency from back xxup local exchange ? \n▁ xxfld 3 xxmaj xxunk xxfld 4 https : / / travel.stackexchange.com / users / xxunk xxfld 5 xxmaj think no seen able xxmaj tanzania , xxmaj . but i have been told xxmaj a few places in xxmaj tanzania . have been to find a online usd in xxmaj xxup of is you the usd . \n\n it i i n't you as the . but . are be a or two of in from . \n▁ xxfld 6 xxmaj xxfld 7 https : / / travel.stackexchange.com / users / xxunk xxfld 8 http : / / travel.stackexchange.com / questions / xxunk 3 3 xxunk / how - it - possible - to - withdraw - usd - from - tanzania - in - tanzania xx

In [ ]:
learn.save_encoder('enc1')

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
